In [ ]:
import re
import json
from datetime import date, datetime

import pandas as pd
import numpy as np
from pathlib import Path

from sqlalchemy import MetaData, func, and_, distinct, tuple_
from db import init_db, get_db
from orm import Base
from orm import Crime

# For simplicity we'll create a local SQLite database for our analysis
- You can use something like DBeaver to connect to and query your database with a SQL editor, by pointing the connection to the file that will be created in your cwd
- You'll need to delete this file when you edit/modify your ORM, so the table can be re-created

In [ ]:
conn_kwargs = {
    'echo': False # can turn this off to ignore all the background DB noise
}
init_db(**conn_kwargs)

### <font color="red"> THIS WILL NOT WORK UNTIL YOU FIX THE orm.py FILE! See Below</font>

In [ ]:
db = get_db()
Base.metadata.create_all(bind=db.engine)

## Some notes on getting this to work
- We needed to add the '--user' flag to the command to create the kernel in our jupyter notebooks from the venv we created for this project. <br>
Our command: python3 -m ipykernel install --user --name pairin_kernel --display-name "PAIRIN_Kernel"
- next, orm would not import until data types were set in the orm.py file for the Crime class. <br>
We used 'String' types for everything on the first pass.

## Let's start with our ORM
- what exploratory data analysis can we do on this to figure out what kind of data we have and what SQL data types and sizes we should use to store the data?  Include whatever you might do to inspect the data and some notes on your thoughts
* we could look at the unique values for each column in the database.
- we need to add a "created_at" column to the dataset
* For Reference: https://stackoverflow.com/questions/7300948/add-column-to-sqlalchemy-table .  It sounds like there a many ways to do this in sqlalchemy, but no method seems to be the accepted or recommended method to do this. In this case, I will try adding the column to the dataframe
- if we had too many rows to open the file in memory, what would you do?
* Generally, I woulld follow the steps used in this example. Work with a large, but usable, chunk of the data and develop our methods using this subsample of data. Once I had ironed out most of the kinks with this data set, I would then probably move to spark and/or databricks and use a scalable system to work with the whole data set. If I can't get spark to work, then I would look at other methods for streaming data through my database builder. If that doesn't work, then I would probably just iterate over the dataset subsample by subsample to get the database built.
- if you were certain about the data types in advance, how could you use that knowledge while loading the file?
* it would make building the orm.py file easier. Also, we can use the data types to set data types in the pd.readcsv command. This could reduce overhead on the process and allow us to work with larger subsets of the entire data set.
- once you know what the pandas data types should be, how do you convert the columns to the correct types before importing?
* https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html - dtypeType name or dict of column -> type, optional Data type for data or columns. E.g. {‘a’: np.float64, ‘b’: np.int32, ‘c’: ‘Int64’} Use str or object together with suitable na_values settings to preserve and not interpret dtype. If converters are specified, they will be applied INSTEAD of dtype conversion.
- re: ORM, don't worry about configuring indexes, keys or things like this, but feel free to describe what columns you might consider for indexing based on how you might expect to use the data and why?
* It looks like lsoa_code could be used as an index for the main table.
- is there a compelling case to de-normalize and create multiple tables with foreign keys?  Don't worry about implemeting that, just walk me through your thoughts?
* It looks like borough could be denormalized to save space. Also, it already looks like this table might be from a set of de-normalized tables as there should be other tables with more details on these events in a data architecture. This table is already pretty small, so there may not be too much value in de-normalizing this data set, unless space is at a premium or the data set is unmanageably large.

In [ ]:
data_path = Path('data')
file_name = 'london_crime_by_lsoa.csv'

# we'll take just the first million rows to speed things up
df = pd.read_csv(data_path/file_name, sep=',', dtype='string', nrows=1_000_000)
print(f'We have {len(df)} records')

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df.describe()

In [ ]:
df['value'].unique()

In [ ]:
df.lsoa_code.str.len().max()

In [ ]:
df.borough.str.len().max()

print('max string length of columns in database')
print(df.lsoa_code.str.len().max(), 'lsoa_code')
print(df.borough.str.len().max(), 'borough')
print(df.major_category.str.len().max(), 'major_category')
print(df.minor_category.str.len().max(), 'minor_category')
print(df.value.str.len().max(), 'value')
print('')
print('Looks like String lengths of 50 should be sufficient.')

In [ ]:
# add a created_at column with current timestamp, or implement your orm to do it for you
# df.columns
df['created_at'] = datetime.now()
df.head()

In [ ]:
# we probably need to convert some pandas data types away from string before importing?
# https://pandas.pydata.org/docs/reference/api/pandas.to_numeric.html
# df.dtypes
df.value = pd.to_numeric(df.value, downcast='integer')
df.year = pd.to_numeric(df.year, downcast='integer')
df.month = pd.to_numeric(df.month, downcast='integer')
df.dtypes

In [ ]:
# update your ORM file to add in appropriate column types
# check orm.py

In [ ]:
# once you have your data types sorted and ORM implemented, you can use this to import the data
def bulk_load(df: pd.DataFrame) -> None:
    chunk_size = 100_000
    for idx in range(0, len(df), chunk_size):        
        print(f'Inserting chunk {idx}:{idx+chunk_size}')

        with get_db().session_scope() as session:

            cols = [
                'lsoa_code', 'borough', 'major_category', 'minor_category',
                'value', 'year', 'month', 'created_at'
            ]         

            records = df[idx:idx+chunk_size][cols].astype(object).to_dict(orient='records')
            session.bulk_insert_mappings(Crime, records, render_nulls=True)

In [ ]:
bulk_load(df)

## Questions to answer, for now ignore the database, and show me your pandas code for the following
- Overall, which borough had the most crime?
- How many major categories of crime are there?
- How many distinct combinations of (major_category, minor_category) are there?
- What were the top 5 major categories of crime?
- What was the count of major cat Burglary for Croydon in December 2014?
- Show me total crime counts per year
- Which borough had to biggest increase in overall crime from 2008 vs 2016

In [ ]:
df.head()

In [ ]:
# Overall, which borough had the most crime?

In [ ]:
# https://pandas.pydata.org/docs/reference/api/pandas.pivot_table.html
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sort_values.html
borough_crime = pd.pivot_table(df, values='value', index=['borough'], aggfunc=np.sum)
print(borough_crime.sort_values(by=['value'], ascending=False))
print('')
print('Westminster is the borough with the most crime')

In [ ]:
# How many major categories of crime are there?

In [ ]:
print(df['major_category'].unique())
print('')
print('there are 9 major categories of crime')

In [ ]:
# How many distinct combinations of (major_category, minor_category) are there?

In [ ]:
print(df.groupby(['major_category', 'minor_category']).size())
print('')
# print(df.drop_duplicates(subset=['major_category', 'minor_category']))
print('there are 32 distinct combinations of (major_category, minor_category)')

In [ ]:
# What were the top 5 major categories of crime?

In [ ]:
top_major = pd.pivot_table(df, values='value', index=['major_category'], aggfunc=np.sum)
print(top_major.sort_values(by=['value'], ascending=False).head(5))
print('')
print('the above are the top 5 major cateroies of crime')

In [ ]:
# What was the count of Burglary for Croydon in 2014?

In [ ]:
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.loc.html
answer = df.loc[((df.major_category == 'Burglary') & (df.borough == 'Croydon') & (df.year == 2014))].shape[0]
print('the count of Burglary for Croydon in 2014 is', answer)

In [ ]:
# Show me total crime counts per year

In [ ]:
crime_year = pd.pivot_table(df, values='value', index=['year'], aggfunc=np.sum)
crime_year

In [ ]:
# Which borough had to biggest increase in crime from 2008 vs 2016

In [ ]:
crime_borough_year = pd.pivot_table(df, values='value', index=['borough', 'year'], aggfunc=np.sum)
crime_borough_2008 = pd.pivot_table(df.loc[df.year == 2008], values='value', index=['borough'], aggfunc=np.sum) 
crime_borough_2016 = pd.pivot_table(df.loc[df.year == 2016], values='value', index=['borough'], aggfunc=np.sum)
crime_borough_group = df.loc[((df.year == 2008) | (df.year == 2016))].groupby(['borough', 'year']).agg({'value' : 'sum'})
# crime_borough_group
take_diff = lambda s1, s2: s1-s2
crime_borough_diff = crime_borough_2016.combine(crime_borough_2008, take_diff)
crime_borough_diff.sort_values(by='value', ascending=False)

In [ ]:
crime_borough_west = df.loc[(((df.year == 2008) | (df.year == 2016)) & (df.borough == 'Westminster'))].groupby(['borough', 'year']).agg({'value' : 'sum'})
crime_borough_west

In [ ]:
print('Which borough had to biggest increase in crime from 2008 vs 2016?')
print('')
print('answer: Westminster')

## OK, same questions to answer, but now let's pretend our dataset is too big for memory, so we need to leverage the database... If you're not familiar with SQLAlchemy, that's ok, just provide the raw SQL to accomplish the task.   You can connect to your database using something like DBeaver https://dbeaver.io/download/ by just pointing it at the crime.db file in you cwd
- Overall, which borough had the most crime?
- How many major categories of crime are there?
- How many distinct combinations of (major_category, minor_category) are there?
- What were the top 5 major categories of crime?
- What was the count of major category Burglary for Croydon in December 2014?
- Show me total crime counts per year
- Which borough had to biggest increase in overall crime from 2008 vs 2016

## For the first one, I'll show the SQL / SQLAlchemy

- Overall, which borough had the most crime?

Raw SQL
```SQL
select 
	borough, sum(value) as total_crimes
from
	crimes
group by
	borough
order by
	sum(value) desc
limit 1
```

Python SQLAlchemy
```python
with get_db().session_scope() as session:

    qry = session.query(
                Crime.borough, func.sum(Crime.value)) \
            .group_by(Crime.borough) \
            .order_by(func.sum(Crime.value).desc()) \
            .limit(1)
    
    print(qry)
    
    df =  pd.read_sql_query(qry.statement, session.bind)
        
df
```

In [ ]:
# How many major categories of crime are there?

In [ ]:
# How many distinct combinations of (major_category, minor_category) are there?

In [ ]:
# What were the top 5 major categories of crime?

In [ ]:
# What was the count of Burglary for Croydon in 2014?

In [ ]:
# Show me total crime counts per year

In [ ]:
# Which borough had to biggest increase in sum total crime from 2008 vs 2016
# this one can be tricky in SQLAlchemy, so raw SQL would suffice if you want